# RAG 시스템 보기


In [1]:
import warnings
from retrieval import load_vectorstore_retriever_embeddings, rag
from model import llm_load
import json

warnings.filterwarnings("ignore")

# 1. 벡터스토어 로드
vectorstore, retriever, embeddings = load_vectorstore_retriever_embeddings("RAG/vectorDB")

# 2. LLM 한 번만 로드 (재사용)
llm = llm_load()

/home/sehan/miniconda/envs/knowledge_conflict/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


벡터스토어 로드 중
검색기 로드 중
검색기 로드 완료


Loading checkpoint shards: 100%|██████████| 5/5 [00:47<00:00,  9.41s/it]


In [2]:
#3. test 데이터셋 가져오기
with open("popqa_dataset/qa_dataset.json", "r") as f:
    test_data = json.load(f)

for item in test_data[:5]:
    question = item["question"]
    print(f"Question: {question}")
    answer = item["answers"]
    print(f"Answers: {answer}\n\n")

Question: What is George Rankin's occupation?
Answers: ["politician", "political leader", "political figure", "polit.", "pol"]


Question: What is John Mayne's occupation?
Answers: ["journalist", "journo", "journalists"]


Question: What is Henry Feilden's occupation?
Answers: ["politician", "political leader", "political figure", "polit.", "pol"]


Question: What is Kathy Saltzman's occupation?
Answers: ["politician", "political leader", "political figure", "polit.", "pol"]


Question: What is Eleanor Davis's occupation?
Answers: ["cartoonist", "graphic artist", "animator", "illustrator"]




In [ ]:
# 3. RAG 실행
import os

output_file = "output_with_base_rag.json"

if not os.path.exists(output_file):
    results = []
    for item in test_data[:]:
        question = item["question"]
        rag_output = rag(vectorstore, question, llm)
        answer = item["answers"]
        
        results.append({
            "Question": question,
            "Answer": rag_output['answer'],
            "Ground_Truth": answer
        })
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

Question: What is George Rankin's occupation?
RAG 실행 시작
vectorstore : <langchain_community.vectorstores.faiss.FAISS object at 0x7fab89d23f50>
LLM 응답 생성 시작
Answer:  I don't know. None of the provided contexts mention a person named George Rankin.
Gold_Answers: ["politician", "political leader", "political figure", "polit.", "pol"]


Question: What is John Mayne's occupation?
RAG 실행 시작
vectorstore : <langchain_community.vectorstores.faiss.FAISS object at 0x7fab89d23f50>
LLM 응답 생성 시작
Answer:  John Mayne is a solicitor.
Gold_Answers: ["journalist", "journo", "journalists"]


Question: What is Henry Feilden's occupation?
RAG 실행 시작
vectorstore : <langchain_community.vectorstores.faiss.FAISS object at 0x7fab89d23f50>
LLM 응답 생성 시작
Answer:  I don't know.
Gold_Answers: ["politician", "political leader", "political figure", "polit.", "pol"]


Question: What is Kathy Saltzman's occupation?
RAG 실행 시작
vectorstore : <langchain_community.vectorstores.faiss.FAISS object at 0x7fab89d23f50>
LLM 응답 생성 시작


KeyboardInterrupt: 

# RAG 성능이 너무 낮아서 테스트 쿼리 한 번 해봄

Query 1: What is George Rankin's occupation?

[Result 1] (score: 0.2973)
Title: Billy Rankin (guitarist)
URL: https://en.wikipedia.org/wiki?curid=24221919
Content preview: William Rankin (born 25 April 1959) is a Scottish guitarist and singer active in the 1980s and 1990s. Best known for his work with Nazareth (1980–83 and 1990–94), he also had a successful solo career, including a US hit single "Baby Come Back".
Rankin was part of multimedia group "Team Rock" prior t...
--------------------------------------------------------------------------------

[Result 2] (score: 0.3025)
Title: Arthur Rankin (actor)
URL: https://en.wikipedia.org/wiki?curid=60861474
Content preview: Arthur Gardner Rankin (August 30, 1895 – March 23, 1947) was an American film actor.
Personal life.
Arthur Rankin was born in New York, the son of actress Phyllis Rankin. She later married actor Harry Davenport, who adopted Arthur. He was the nephew of Mr. and Mrs. Sidney Drew. Rankin served in the ...
---------------

KeyboardInterrupt: 